## Nulladrendű rezolúció

In [1]:
class Expr:
    def __init__(self,lhs,rhs,op):
        self.lhs,self.rhs,self.op = lhs,rhs,op
    
    def __or__(self,rhs):
        if rhs.op == 'and':
            return (self | rhs.lhs) & (self | rhs.rhs)
        if self.op == 'and':
            return (self.lhs | rhs) & (self.rhs | rhs)
        else:
            return Expr(self,rhs,'or')
    
    def __and__(self,rhs):
        return Expr(self,rhs,'and')
    
    def __invert__(self):
        if self.op == 'not':
            return self.lhs
        elif self.op == 'and':
            return (~self.lhs) | (~self.rhs)
        elif self.op == 'or':
            return (~self.lhs) & (~self.rhs)
        else:
            return Expr(self,None,'not')
    
    def __gt__(self,rhs):
        return ~self | rhs
    
    def __eq__(self,rhs):
        return (self > rhs) & (rhs > self)
    
    def kloz(self,arr):
        if self.op == 'or':
            self.lhs.kloz(arr)
            self.rhs.kloz(arr)
        elif self.op == 'and':
            self.lhs.kloz(arr)
            arr += [set()]
            self.rhs.kloz(arr)
        elif self.op == 'not':
            if self.lhs.lhs in arr[-1]:
                arr.pop()
            elif '¬' + self.lhs.lhs not in arr[-1]:
                arr[-1].update(['¬' + self.lhs.lhs])
        else:
            if '¬' + self.lhs in arr[-1]:
                arr.pop()
            elif self.lhs not in arr[-1]:
                arr[-1].update([self.lhs])
        return arr

def valtozok(s):
    return [Expr(i,None,'var') for i in s]

In [2]:
def egyesit(ek,k):
    if ek[0] == '¬':
        nek = ek[1:]
    else:
        nek = '¬' + ek
    if nek in k:
        k.remove(nek)

def rezolucio(klozok):
    kc = klozok.copy()
    while len(kc)>0:
        print(kc)
        kc.sort(key=len,reverse=True)
        ek = kc.pop()
        if len(ek) == 0:
            return True
        elif len(ek) > 1:
            return False
        else:
            ek = ek.pop()
            for k in kc:
                egyesit(ek,k)

**Példa.** (ezt pár éve ZH példának adtam fel az egyetemen, persze papíron)

Charlie Firpo a következőket állítja:

- Ha alszom és nem kellő tapintattal ébresztenek fel, elsírom magam: $(A \wedge \neg T \wedge E) \rightarrow S$
- A sírással felizgatom magam, és izgalmamban fel kell kelnem: $(S \rightarrow I) \wedge (I \rightarrow K)$
- Ha felkelek, bal lábbal kelek: $K \rightarrow B$

Hozzuk klóz formára az állításokat, majd bizonyítsuk be nulladrendű rezolúcióval, hogy ha Charlie nem bal lábbal ébredt, akkor vagy tapintattal ébresztették, vagy nem aludt:

$(\neg B \wedge E) \rightarrow (\neg A \vee T)$

In [3]:
A,T,E,S,I,K,B = valtozok('ATESIKB')

all1 = (A & ~T & E) > S
all2 = (S > I) & (I > K)
all3 = K > B
biz  = (~B & E) > (~A | T)

klozok = (all1 & all2 & all3 & ~biz).kloz([set()])
rezolucio(klozok)

[{'T', 'S', '¬E', '¬A'}, {'I', '¬S'}, {'¬I', 'K'}, {'B', '¬K'}, {'¬B'}, {'E'}, {'A'}, {'¬T'}]
[{'S', '¬E', '¬A'}, {'I', '¬S'}, {'¬I', 'K'}, {'B', '¬K'}, {'¬B'}, {'E'}, {'A'}]
[{'S', '¬E'}, {'I', '¬S'}, {'¬I', 'K'}, {'B', '¬K'}, {'¬B'}, {'E'}]
[{'S'}, {'I', '¬S'}, {'¬I', 'K'}, {'B', '¬K'}, {'¬B'}]
[{'I', '¬S'}, {'¬I', 'K'}, {'¬K'}, {'S'}]
[{'I'}, {'¬I', 'K'}, {'¬K'}]
[{'¬I'}, {'I'}]
[set()]


True